## Import Packages and Set Directory

In [3]:
import pandas as pd
import numpy as np
import sys
import argparse
import os
os.chdir('/Users/abc6435/Desktop/RABvcfs')

## Functions

In [30]:
#Import Population File
pop_file="/Users/abc6435/Desktop/RABvcfs_sims/pops.txt"
pops = pd.read_csv(pop_file, sep="\t", header=None)
popA = pops[pops[0]=="popA"][1].astype(str)
popB = pops[pops[0]=="popB"][1].astype(str)

#Import VCF
vcf_file="/Users/abc6435/Desktop/RABvcfs_sims/rec_test.vcf"
with open(vcf_file) as file:
    for line in file:
        if line.startswith('#CHROM'):
            cols = line.lstrip("#").strip().split("\t")
data = pd.read_csv(vcf_file, sep='\t', comment='#', names=cols)

#Extract Genotype Information
loci = data[['CHROM','POS','REF','ALT']]
samples = data.drop(columns=['CHROM', 'POS', 'REF', 'ALT','ID','QUAL','FILTER','INFO','FORMAT'])
samples = samples.apply(lambda col: col.str.split(":").str[0])
genotypes = pd.concat([loci,samples], axis=1)

#Count Genotypes
genotypes['popA_0'] = 0
genotypes['popA_1'] = 0
genotypes['popB_0'] = 0
genotypes['popB_1'] = 0

for sample in popA:
    for row in range(len(genotypes)):
        if genotypes.loc[row, sample] == '0/0' or genotypes.loc[row, sample] == '0|0':
            genotypes.loc[row, 'popA_0'] += 2
        if genotypes.loc[row, sample] == '1/1' or genotypes.loc[row, sample] == '1|1':
            genotypes.loc[row, 'popA_1'] += 2
        if genotypes.loc[row, sample] == '0/1' or genotypes.loc[row, sample] == '0|1':
            genotypes.loc[row, 'popA_0'] += 1
            genotypes.loc[row, 'popA_1'] += 1
        if genotypes.loc[row, sample] == '1/0' or genotypes.loc[row, sample] == '1|0':
            genotypes.loc[row, 'popA_0'] += 1
            genotypes.loc[row, 'popA_1'] += 1
                
for sample in popB:
    for row in range(len(genotypes)):
        if genotypes.loc[row, sample] == '0/0' or genotypes.loc[row, sample] == '0|0':
            genotypes.loc[row, 'popB_0'] += 2
        if genotypes.loc[row, sample] == '1/1' or genotypes.loc[row, sample] == '1|1':
            genotypes.loc[row, 'popB_1'] += 2
        if genotypes.loc[row, sample] == '0/1' or genotypes.loc[row, sample] == '0|1':
            genotypes.loc[row, 'popB_0'] += 1
            genotypes.loc[row, 'popB_1'] += 1
        if genotypes.loc[row, sample] == '1/0' or genotypes.loc[row, sample] == '1|0':
            genotypes.loc[row, 'popB_0'] += 1
            genotypes.loc[row, 'popB_1'] += 1
genotypes

,CHROM,POS,REF,ALT,29779,383194,383202,383205,507264,507265,...,183195332,183194861,183195321,183195304,183195326,183195312,popA_0,popA_1,popB_0,popB_1
0,20,111,A,T,./.,./.,./.,./.,./.,./.,...,0|0,0|1,0|0,0|0,0|0,0|0,0,0,13,1
1,20,667,A,T,0|1,1|0,1|0,0|1,1|1,1|1,...,0|1,0|0,0|0,0|0,1|0,0|0,4,8,12,2
2,20,748,A,T,./.,./.,./.,./.,./.,./.,...,1|0,1|0,1|1,1|1,0|1,0|0,0,0,7,7
3,20,1363,A,T,./.,./.,./.,./.,./.,./.,...,1|0,1|0,1|1,1|1,0|1,0|0,0,0,7,7
4,20,1504,A,T,1|0,0|1,0|1,1|0,0|0,0|0,...,0|0,0|1,0|0,0|0,0|0,1|1,8,4,9,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34484,20,14969815,A,T,1|1,0|0,0|1,0|1,1|0,0|1,...,0|0,1|0,1|0,0|0,0|0,0|0,6,6,12,2
34485,20,14993157,A,T,./.,./.,./.,./.,./.,./.,...,0|0,0|1,0|0,0|0,0|0,0|0,0,0,13,1
34486,20,15014437,A,T,0|0,1|0,0|0,0|0,0|0,0|0,...,1|0,0|1,0|0,1|1,0|0,1|0,11,1,9,5
34487,20,15014672,A,T,0|0,1|1,1|0,1|0,0|1,1|0,...,1|1,0|1,0|1,1|1,1|1,1|1,6,6,2,12


In [35]:
#Simulated Data Boolean
simulated_vcf = True

#Calculate Frequencies (Empirical VCF)
genotypes['f_popA_0'] = genotypes['popA_0']/(genotypes['popA_0'] + genotypes['popA_1'])
genotypes['f_popA_1'] = genotypes['popA_1']/(genotypes['popA_0'] + genotypes['popA_1'])
genotypes['f_popB_0'] = genotypes['popB_0']/(genotypes['popB_0'] + genotypes['popB_1'])
genotypes['f_popB_1'] = genotypes['popB_1']/(genotypes['popB_0'] + genotypes['popB_1'])
genotypes['nA'] = (genotypes['popA_0'] + genotypes['popA_1']) / 2
genotypes['nB'] = (genotypes['popB_0'] + genotypes['popB_1']) / 2
freq = genotypes[['CHROM','POS','REF','ALT',
                      'f_popA_1','f_popB_1','nA','nB']].dropna(axis="rows")

#Adjust Frequencies for sites lost/gained (Simulated VCF)
if simulated_vcf: 
    miss_popA = (genotypes['popA_0'] == 0) & (genotypes['popA_1'] == 0)
    genotypes.loc[miss_popA, 'f_popA_0'] = 0
    genotypes.loc[miss_popA, 'f_popA_1'] = 0
    miss_popB = (genotypes['popB_0'] == 0) & (genotypes['popB_1'] == 0)
    genotypes.loc[miss_popB, 'f_popB_0'] = 0
    genotypes.loc[miss_popB, 'f_popB_1'] = 0
    genotypes.loc[miss_popA, 'nA'] = len(popA)
    genotypes.loc[miss_popA, 'nB'] = len(popB)
    freq = genotypes[['CHROM','POS','REF','ALT','f_popA_1','f_popB_1','nA','nB']]
freq



,CHROM,POS,REF,ALT,f_popA_1,f_popB_1,nA,nB
0,20,111,A,T,0.000000,0.071429,6.0,7.0
1,20,667,A,T,0.666667,0.142857,6.0,7.0
2,20,748,A,T,0.000000,0.500000,6.0,7.0
3,20,1363,A,T,0.000000,0.500000,6.0,7.0
4,20,1504,A,T,0.333333,0.357143,6.0,7.0
...,...,...,...,...,...,...,...,...
34484,20,14969815,A,T,0.500000,0.142857,6.0,7.0
34485,20,14993157,A,T,0.000000,0.071429,6.0,7.0
34486,20,15014437,A,T,0.083333,0.357143,6.0,7.0
34487,20,15014672,A,T,0.500000,0.857143,6.0,7.0


In [9]:
#Define readvcf()
def readvcf(vcf_file, pop_file, simulated_vcf=False):
    #Import Population File
    pops = pd.read_csv(pop_file, sep="\t", header=None)
    popA = pops[pops[0]=="popA"][1].astype(str)
    popB = pops[pops[0]=="popB"][1].astype(str)

    #Import VCF
    with open(vcf_file) as file:
        for line in file:
            if line.startswith('#CHROM'):
                cols = line.lstrip("#").strip().split("\t")
    data = pd.read_csv(vcf_file, sep='\t', comment='#', names=cols)

    #Extract Genotype Information
    loci = data[['CHROM','POS','REF','ALT']]
    samples = data.drop(columns=['CHROM', 'POS', 'REF', 'ALT','ID','QUAL','FILTER','INFO','FORMAT'])
    samples = samples.apply(lambda col: col.str.split(":").str[0])
    genotypes = pd.concat([loci,samples], axis=1)

    #Count Genotypes
    genotypes['popA_0'] = 0
    genotypes['popA_1'] = 0
    genotypes['popB_0'] = 0
    genotypes['popB_1'] = 0

    for sample in popA:
        for row in range(len(genotypes)):
            if genotypes.loc[row, sample] == '0/0' or genotypes.loc[row, sample] == '0|0':
                genotypes.loc[row, 'popA_0'] += 2
            if genotypes.loc[row, sample] == '1/1' or genotypes.loc[row, sample] == '1|1':
                genotypes.loc[row, 'popA_1'] += 2
            if genotypes.loc[row, sample] == '0/1' or genotypes.loc[row, sample] == '0|1':
                genotypes.loc[row, 'popA_0'] += 1
                genotypes.loc[row, 'popA_1'] += 1
            if genotypes.loc[row, sample] == '1/0' or genotypes.loc[row, sample] == '1|0':
                genotypes.loc[row, 'popA_0'] += 1
                genotypes.loc[row, 'popA_1'] += 1
                
    for sample in popB:
        for row in range(len(genotypes)):
            if genotypes.loc[row, sample] == '0/0' or genotypes.loc[row, sample] == '0|0':
                genotypes.loc[row, 'popB_0'] += 2
            if genotypes.loc[row, sample] == '1/1' or genotypes.loc[row, sample] == '1|1':
                genotypes.loc[row, 'popB_1'] += 2
            if genotypes.loc[row, sample] == '0/1' or genotypes.loc[row, sample] == '0|1':
                genotypes.loc[row, 'popB_0'] += 1
                genotypes.loc[row, 'popB_1'] += 1
            if genotypes.loc[row, sample] == '1/0' or genotypes.loc[row, sample] == '1|0':
                genotypes.loc[row, 'popB_0'] += 1
                genotypes.loc[row, 'popB_1'] += 1

    #Calculate Frequencies (Empirical VCF)
    genotypes['f_popA_0'] = genotypes['popA_0']/(genotypes['popA_0'] + genotypes['popA_1'])
    genotypes['f_popA_1'] = genotypes['popA_1']/(genotypes['popA_0'] + genotypes['popA_1'])
    genotypes['f_popB_0'] = genotypes['popB_0']/(genotypes['popB_0'] + genotypes['popB_1'])
    genotypes['f_popB_1'] = genotypes['popB_1']/(genotypes['popB_0'] + genotypes['popB_1'])
    genotypes['nA'] = (genotypes['popA_0'] + genotypes['popA_1']) / 2
    genotypes['nB'] = (genotypes['popB_0'] + genotypes['popB_1']) / 2
    freq = genotypes[['CHROM','POS','REF','ALT',
                      'f_popA_1','f_popB_1','nA','nB']].dropna(axis="rows")
    freq = freq[(freq["nA"] > 1) & (freq["nB"] > 1)]

    #Adjust Frequencies for sites lost/gained (Simulated VCF)
    if simulated_vcf: 
        miss_popA = (genotypes['popA_0'] == 0) & (genotypes['popA_1'] == 0)
        genotypes.loc[miss_popA, 'f_popA_0'] = 0
        genotypes.loc[miss_popA, 'f_popA_1'] = 0
        miss_popB = (genotypes['popB_0'] == 0) & (genotypes['popB_1'] == 0)
        genotypes.loc[miss_popB, 'f_popB_0'] = 0
        genotypes.loc[miss_popB, 'f_popB_1'] = 0
        genotypes.loc[miss_popA, 'nA'] = len(popA)
        genotypes.loc[miss_popA, 'nB'] = len(popB)
        freq = genotypes[['CHROM','POS','REF','ALT','f_popA_1','f_popB_1','nA','nB']]
    return freq

#Define importsites()
def importsites(neutral_file, mutation_file, derived_sites):
    neutral = pd.read_csv(neutral_file, sep='\t', header=(0))
    mutation = pd.read_csv(mutation_file, sep='\t', header=(0))
    neutral.rename(columns={'chromo':'CHROM', 'position':'POS'}, inplace=True)
    mutation.rename(columns={'chromo':'CHROM', 'position':'POS'}, inplace=True)
    neu_der = pd.merge(derived_sites, neutral, on=['CHROM','POS'],how='inner', indicator=True)
    mut_der = pd.merge(derived_sites, mutation, on=['CHROM','POS'],how='inner', indicator=True)
    neu_der = neu_der[neu_der['_merge']=='both'].drop(columns=['_merge'])
    mut_der = mut_der[mut_der['_merge']=='both'].drop(columns=['_merge'])
    #Report Number of Sites
    print("Number of derived mutations =", len(mut_der))
    return neu_der, mut_der

#Define calcRAB()
def calcRAB(neu_der, mut_der, seed):
    np.random.seed(seed)
    index1=np.random.permutation(len(neu_der))[:10000]
    neu1=neu_der.iloc[index1]
    f_AD = mut_der['f_popA_1']
    f_BD = mut_der['f_popB_1']
    f_AN = neu1['f_popA_1']
    f_BN = neu1['f_popB_1']
    LAB = sum(f_AD*(1-f_BD))/sum(f_AN*(1-f_BN))
    LBA = sum(f_BD*(1-f_AD))/sum(f_BN*(1-f_AN))
    RAB = LAB/LBA
    return RAB

#Define calcRAB_neu()
def calcRAB_neu(neu_der, seed):
    np.random.seed(seed)
    index1=np.random.permutation(len(neu_der))[:10000]
    neu1=neu_der.iloc[index1]
    index2=np.random.permutation(len(neu_der))[:10000]
    neu2=neu_der.iloc[index2]
    f_AD = neu1['f_popA_1']
    f_BD = neu1['f_popB_1']
    f_AN = neu2['f_popA_1']
    f_BN = neu2['f_popB_1']
    LAB = sum(f_AD*(1-f_BD))/sum(f_AN*(1-f_BN))
    LBA = sum(f_BD*(1-f_AD))/sum(f_BN*(1-f_AN))
    RAB_neu = LAB/LBA
    return RAB_neu

#Define calcRAB_subs()
def calcRAB_sub(neu_sub, mut_sub):
    index1=np.random.permutation(len(neu_sub))[:10000]
    neu1=neu_sub.iloc[index1]
    f_AD = mut_sub['f_popA_1']
    f_BD = mut_sub['f_popB_1']
    f_AN = neu1['f_popA_1']
    f_BN = neu1['f_popB_1']
    LAB = sum(f_AD*(1-f_BD))/sum(f_AN*(1-f_BN))
    LBA = sum(f_BD*(1-f_AD))/sum(f_BN*(1-f_AN))
    RAB_sub = LAB/LBA
    return RAB_sub

#Define samplesites()
def samplesites(sites, psites):
    nsites = int(round(len(sites) * psites))
    indices = np.random.permutation(len(sites))[:nsites]
    subsamp = sites.iloc[indices]
    return subsamp

#Define jackknife()
def jackknife(neu_der, mut_der, psites, iter, jackknife=False):
    if jackknife:
        jx = []
        for i in range(iter):
            neu_sub = samplesites(neu_der, psites)
            mut_sub = samplesites(mut_der, psites)
            jx.append(calcRAB_sub(neu_sub, mut_sub))
    return np.array(jx)
    

## Run

In [ ]:
der = readvcf("/Users/abc6435/Desktop/RABvcfs_sims/rec_test.vcf", 
        "/Users/abc6435/Desktop/RABvcfs_sims/pops.txt",
        simulated_vcf=True)

neutral, mutation = importsites('/Users/abc6435/Desktop/RABvcfs_sims/intergenic20.txt',
                                '/Users/abc6435/Desktop/RABvcfs_sims/neutral20.txt', der)
print("calcRAB", calcRAB(neutral, mutation, 20))
print("calcRAB_neu",calcRAB_neu(neutral, 20))
jx_array=jackknife(neutral, mutation, 0.30, 5)
print("jackknife", jx_array)
np.percentile(jx_array, [2.5, 97.5])

Number of derived mutations = 849


## Terminal

In [ ]:
source ~/RABvcfs_env/bin/activate
python3 /Users/abc6435/Desktop/KROH/scripts/analysis/slim/test/RABvcfs_sims.py --vcf "rec_test.vcf" --pop pops.txt --fileN intergenic20.txt --fileM neutral20.txt --seed 34 --psites 0.30 --iter 5 --simulated_vcf

calcRAB 0.9821766816874066
calcRAB_neu 1.021948329297477
jackknife [0.91832788 1.07702961 0.84243057 1.08278136 0.90063503]


array([0.84825102, 1.08220619])